## MULTI_LABEL TEXT CLASSIFICATION POUR STACK OVERFLOW  PREDICTION DE TAG

In [52]:
#!pip install -U scikit-learn
#!pip install wordcloud
#!pip install gensim

In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
import joblib

In [2]:
import streamlit

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)

In [4]:
import os

In [5]:
#os.chdir("/home/sacko/Documents/Proje5_NLP")

In [6]:
#os.getcwd()

In [113]:
df_cleaned = pd.read_csv('/home/alseny/Documents/ModelFinal/QueryResultCleaneed.csv')
df_cleaned.head()

,Tags,Posts,PostsClean
0,['c#'],How to convert Decimal to Double in C#? I want...,convert decimal double want assign decimal var...
1,"['c#', '.net']",How do I calculate someone's age based on a Da...,calculate someone based datetime type birthday...
2,['c#'],Calculate relative time in C# Given a specific...,calculate relative time given specific value d...
3,['.net'],Difference between Math.Floor() and Math.Trunc...,difference mathfloor mathtruncate difference
4,['c#'],Filling a DataSet or a DataTable from a LINQ q...,filling dataset datatable linq query result ex...


In [114]:
df_clean = df_cleaned.copy()

In [115]:
df_clean.Tags[0]

"['c#']"

In [116]:
import ast
df_clean.Tags = df_clean.Tags.apply(lambda x : ast.literal_eval(x))

In [117]:
df_clean.Tags[0]

['c#']

In [135]:
def print_plot(index):
    example = df_clean[df_clean.index == index][['PostsClean', 'Tags']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])
print_plot(1)

calculate someone based datetime type birthday given representing person birthday calculate year
Tag: ['c#', '.net']


In [119]:
print_plot(6)

check change server table monitor server database change table without using trigger modifying structure database preferred programming environment c
id like able support server 2000 newer application bolton data visualization another company product customer base thousand dont want requirement modify thirdparty vendor table every installation
by change table mean change table data change table structure
ultimately would like change trigger event application instead check change interval

the best course action given requirement trigger schema modification server 2000 2005 seems function tsql plan implement this
every second following query
 
and compare stored value value changed table using query
 
and compare returned checksum stored value
Tag: ['sql', 'sql-server']


In [120]:
df_clean.drop(["Posts"], axis=1, inplace=True)## Suppression de la variable "Posts"

In [121]:
df_clean = df_clean[0:500]

In [122]:
df_clean.shape

(500, 2)

In [123]:
df_clean.head()

,Tags,PostsClean
0,[c#],convert decimal double want assign decimal var...
1,"[c#, .net]",calculate someone based datetime type birthday...
2,[c#],calculate relative time given specific value d...
3,[.net],difference mathfloor mathtruncate difference
4,[c#],filling dataset datatable linq query result ex...


## TF-IDF & SGDClassifier

In [124]:
#conda deactivate
#!pip install tensorflow

In [125]:
X = df_clean['PostsClean']

y = df_clean['Tags']

multilabel_binarizer = MultiLabelBinarizer()

y_target = multilabel_binarizer.fit_transform(y)

# Split into Training and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_target, test_size=0.3, random_state=42)

In [126]:
#y_test = np.reshape(y_test,(-1,1))

#y_train = np.reshape(y_train,(-1,1))

In [127]:
y_train.shape

(350, 10)

In [128]:
# Initizalize the vectorizer with max nr words and ngrams (1: single words, 2: two words in a row)
vectorizer_tfidf = TfidfVectorizer(analyzer ="word", max_features=5000, ngram_range=(1,2))

## TF-IDF & SGDClassifier

In [129]:
rf_classifier_tf = MultiOutputClassifier(SGDClassifier())

In [130]:
model_rf = Pipeline([("vectorizer", vectorizer_tfidf), ("classifier", rf_classifier_tf)])

model_rf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_features=5000, ngram_range=(1, 2))),
                ('classifier',
                 MultiOutputClassifier(estimator=SGDClassifier()))])

In [131]:
predicted_train_tfidf = model_rf.predict(X_train)

accuracy_train_tfidf = accuracy_score(y_train, predicted_train_tfidf)

print('Accuracy Training data: {:.1%}'.format(accuracy_train_tfidf))

predicted_test_tfidf = model_rf.predict(X_test)

accuracy_test_tfidf = accuracy_score(y_test, predicted_test_tfidf)

print('Accuracy Test data: {:.1%}'.format(accuracy_test_tfidf))

print(classification_report(y_test, predicted_test_tfidf))

Accuracy Training data: 100.0%
Accuracy Test data: 22.0%
              precision    recall  f1-score   support

           0       0.57      0.19      0.29        42
           1       0.88      0.41      0.56        17
           2       0.58      0.30      0.40        46
           3       1.00      0.11      0.20         9
           4       1.00      0.50      0.67        12
           5       1.00      0.75      0.86         8
           6       0.17      0.06      0.09        16
           7       1.00      0.57      0.73         7
           8       0.00      0.00      0.00        16
           9       0.67      0.56      0.61        18

   micro avg       0.68      0.30      0.41       191
   macro avg       0.69      0.35      0.44       191
weighted avg       0.61      0.30      0.39       191
 samples avg       0.32      0.32      0.31       191



/home/alseny/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alseny/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [137]:
ex="calculate someone based datetime type birthday given representing person birthday calculate year"

In [148]:
#Exemple de prediction sur un texte
multilabel_binarizer.inverse_transform(model_rf.predict([ex]))

[('.net', 'c#')]

## Sauvegarde du modèle

In [142]:
import pickle as pkl

In [143]:
# saving the model 
pipline_file = open("TagsPredict.pkl", "wb") 
joblib.dump(model_rf, pipline_file) 
pipline_file.close()